# Make clean data

In [473]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

dfBubble = pd.read_csv("Results16/bubbleSort-PC16.csv")
dfBubble.columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

dfReadCSV = pd.read_csv("Results16/readCSV-PC16.csv")
dfReadCSV.columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]


In [474]:
def removeNonsense(df):
    df = df[df["PKG (J)"]>=0]
    df = df[df["CPU (J)"]>=0]
    df = df[df["Time (ms)"]>=0]
    return df

In [475]:
def makeinputColumn(df):
    l = []
    for i in range(len(df)):
        l.append(df.iloc[i]["Test name"].split("-")[0])

    df["Input case"] = l

def makeSizecolumn(df):
    l = []
    for i in range(len(df)):
        l.append(int(df.iloc[i]["Test name"].split("-")[1]))

    df["Size"] = l

In [476]:
def makeinputColumn_readCSV(df):
    l = []
    for i in range(len(df)):
        l.append(df.iloc[i]["Test name"].split("-")[1].lower())
    df["Input case"] = l

def makeSizecolumn_readCSV(df):
    l = []
    for i in range(len(df)):
        l.append(int(df.iloc[i]["Test name"].split("-")[-1]))

    df["Size"] = l

In [477]:
def removeOutliers(df, type_, size, col):
    q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
    q3 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.75)
    IQR = q3 - q1 
    LF = q1 - 1.5 * IQR
    UF = q3 + 1.5 * IQR

    df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]
    df2 = df1[df1["Input case"] == type_][df1["Size"]==size][df1[col] >= LF]
    return df2[col]


In [478]:
def CleanDataframe(df):
    cleanList = []
    for case in ["random", "best", "worst"]:
        if case != "random":
            for size in [25000,50000,75000,100000,200000,300000,400000,500000]:
                cleanPortion = pd.DataFrame({"PKG (J)":[]})
                for col in ["PKG (J)", "CPU (J)", "DRAM (J)","Time (ms)", "Temp (C)"]:
                    cleanPortion[col] = removeOutliers(df, case, size, col)
                
                cleanPortion["Input case"] = [case for _ in range(len(cleanPortion))]
                cleanPortion["Size"] = [int(size) for _ in range(len(cleanPortion))]

                cleanList.append(cleanPortion)
        else:
            for i in range(1,11):
                for size in [25000,50000,75000,100000,200000,300000,400000,500000]:
                    cleanPortion = pd.DataFrame({"PKG (J)":[]})
                    for col in ["PKG (J)", "CPU (J)", "DRAM (J)","Time (ms)", "Temp (C)"]:
                        cleanPortion[col] = removeOutliers(df, case+str(i), size, col)
                
                    cleanPortion["Input case"] = [case+str(i) for _ in range(len(cleanPortion))]
                    cleanPortion["Size"] = [int(size) for _ in range(len(cleanPortion))]
                    cleanList.append(cleanPortion)
    cleanBubble = pd.concat(cleanList)
    return cleanBubble

In [479]:
def prepareDataframe(df, DataOrBase):
    df = removeNonsense(df)

    if DataOrBase == "data":
        makeinputColumn(df)
        makeSizecolumn(df)
    else:
        makeinputColumn_readCSV(dfBubble)
        makeSizecolumn_readCSV(dfBubble)    

    return df


In [480]:
def CleanCSV (df, DataOrBase, name, pcNO):
    ready = prepareDataframe(df, DataOrBase)
    clean = CleanDataframe(ready).dropna().reset_index(drop=True)
    clean.to_csv("CleanData\\"+name+"Clean-PC"+pcNO+".csv", sep=",", index=False)

In [481]:
CleanCSV(dfBubble, "data","Bubble","16")

C:\Users\langk\AppData\Local\Temp\ipykernel_10740\2727353117.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
C:\Users\langk\AppData\Local\Temp\ipykernel_10740\2727353117.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q3 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.75)
C:\Users\langk\AppData\Local\Temp\ipykernel_10740\2727353117.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]
C:\Users\langk\AppData\Local\Temp\ipykernel_10740\2727353117.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
C:\Users\langk\AppData\Local\Temp\ipykernel_10740\2727353117.py:3: UserWarning: Boolean Series key will be reindexed to match 